In [43]:
import pandas as pd
from folium import plugins
import requests

In [27]:
CLIENT_ID = 'FHRGXUUQOWVRRBRFMUOVMI3EPOA54GTYVQJGYWEFKMFYDIDG'
CLIENT_SECRET = '3YTTEIB4LUSPZL4DYXX0FREKHIYOFOWMS5SXTRHAGFBOPTR1'

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "food",
    "near": "Krakow",
    "radius": 4000,
    "limit": 100}

In [28]:
data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [29]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [30]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Krakow', 'Krakow')

In [31]:
d["suggestedBounds"], d["totalResults"]


({'ne': {'lat': 50.07844066002748, 'lng': 19.96527575117025},
  'sw': {'lat': 50.043369873332026, 'lng': 19.901070740353166}},
 203)

In [35]:
d

{'suggestedFilters': {'header': 'Tap to show:',
  'filters': [{'name': 'Open now', 'key': 'openNow'}]},
 'geocode': {'what': '',
  'where': 'krakow',
  'center': {'lat': 50.06143, 'lng': 19.93658},
  'displayString': 'Krakow, Poland',
  'cc': 'PL',
  'geometry': {'bounds': {'ne': {'lat': 50.125746929273035,
     'lng': 20.217017998898445},
    'sw': {'lat': 49.96750603553661, 'lng': 19.793890515115354}}},
  'slug': 'krakow-poland',
  'longId': '72057594041022738'},
 'headerLocation': 'Krakow',
 'headerFullLocation': 'Krakow',
 'headerLocationGranularity': 'city',
 'query': 'food',
 'totalResults': 203,
 'suggestedBounds': {'ne': {'lat': 50.07844066002748,
   'lng': 19.96527575117025},
  'sw': {'lat': 50.043369873332026, 'lng': 19.901070740353166}},
 'groups': [{'type': 'Recommended Places',
   'name': 'recommended',
   'items': [{'reasons': {'count': 0,
      'items': [{'summary': 'This spot is popular',
        'type': 'general',
        'reasonName': 'globalInteractionReason'}]},
   

In [36]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 100


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '597cc55b6dcf041c42573b04',
  'name': 'Pizzatopia',
  'location': {'address': 'Szewska 22',
   'lat': 50.062943238240265,
   'lng': 19.933833687142826,
   'labeledLatLngs': [{'label': 'display',
     'lat': 50.062943238240265,
     'lng': 19.933833687142826}],
   'postalCode': '31-009',
   'cc': 'PL',
   'city': 'Kraków',
   'state': 'Lesser Poland',
   'country': 'Polska',
   'formattedAddress': ['Szewska 22', '31-009 Kraków', 'Polska']},
  'categories': [{'id': '4bf58dd8d48988d1ca941735',
    'name': 'Pizza Place',
    'pluralName': 'Pizza Places',
    'shortName': 'Pizza',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-3-597cc55b6dcf041c42573b04-0'}

In [37]:
del_index = []
no_del_index = []
for i in range(0, len(items)):
    if 'address' in items[i]['venue']['location']:
        no_del_index.append(i)
    else:
        del_index.append(i)   
del_index.sort(reverse=True)

In [38]:
for i in del_index:
    del items[i]

In [39]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address = location["address"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "food", "address", "postalcode", "lat", "lng"])
print("found %i restaurants" % len(df))
df.head()

found 59 restaurants


,uid,name,food,address,postalcode,lat,lng
0,597cc55b6dcf041c42573b04,Pizzatopia,Pizza,Szewska 22,31-009,50.062943,19.933834
1,53e678c5498e35f10c8bcbfa,Bianca Pasta & Wine,Italian,Plac Mariacki 2,31-042,50.061899,19.939741
2,4e6f50cb483bd41a3db4dfa4,MoaBurger,Burgers,Mikołajska 3,31-027,50.061306,19.941310
3,53a1380f498e903f74523f65,Massolit Bakery & Cafe,Bakery,Smoleńsk 17,31-108,50.058975,19.930351
4,56377311498ea3a0befc336c,Bonerowska Gourmet Steak & Fish,Steakhouse,Rynek Główny 42,31-013,50.062246,19.937987


In [40]:
print(df['food'].value_counts())

Café                  13
Restaurant             5
Italian                5
Breakfast              4
Bakery                 4
Pizza                  4
Polish                 3
Vegetarian / Vegan     3
French                 3
Ramen                  2
Steakhouse             2
Burgers                2
Middle Eastern         1
Thai                   1
Sandwiches             1
Creperie               1
Salad                  1
Chinese                1
Falafel                1
American               1
Dumplings              1
Name: food, dtype: int64


In [58]:
krakow_center = d["geocode"]["center"]
krakow_center

{'lat': 50.06143, 'lng': 19.93658}

In [57]:
# create map of Helsinki using latitude and longitude values
map_krakow = folium.Map(location=[krakow_center["lat"], krakow_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_krakow)

add_markers(df)
print(type(df[['lat','lng']]))
hm_data = df[['lat','lng']].values.tolist()
map_krakow.add_child(plugins.HeatMap(hm_data))

map_krakow

<class 'pandas.core.frame.DataFrame'>


In [66]:
# center: {'lat': 50.06143, 'lng': 19.93658}
lat = 50.05524
lng = 19.93358
map_krakow = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our Cafe!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_krakow)
map_krakow